In [1]:
import polars as pl
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from src.config import cfg
from src.data import anime_id_label_encoding, load_data, user_id_label_encoding
from src.seed import seed_everything

pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(50)
pl.Config.set_tbl_cols(100)

seed_everything(cfg.seed)


## 使用する最低限のデータを準備


In [2]:
train_df, test_df, anime_df = load_data()

# testをseen, unseenで分ける
train_user_list = train_df["user_id"].unique().to_list()
seen_test_df = test_df.filter(pl.col("user_id").is_in(train_user_list))
unseen_test_df = test_df.filter(~pl.col("user_id").is_in(train_user_list))


## CV

In [3]:
# seen用のCV
skf = StratifiedKFold(n_splits=cfg.n_splits, shuffle=True, random_state=cfg.seed)

train_df = train_df.with_columns(fold=pl.lit(-1))

for fold, (_, val_index) in enumerate(skf.split(train_df, train_df["user_id"])):
    train_df[val_index, "fold"] = fold

scores_lgb = []
models_lgb = []
feature_importances = []

for fold in range(cfg.n_splits):
    print(f"Training for fold: {fold}...")

    train_data = train_df.filter(pl.col("fold") != fold)
    val_data = train_df.filter(pl.col("fold") == fold)

    features = test_df.columns
    target = "score"


/Users/marumarukun/Documents/compe/atmacup_15/.venv/lib/python3.11/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


## モデル